In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [8]:
#load the saved model
loaded_model = xgb.Booster()
loaded_model.load_model('LeoPayu_BBILAP_60_in_15mob.json')
print("Model Loaded Successfully")

Model Loaded Successfully


In [9]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [10]:
combined_data = model_data.copy()

In [11]:
combined_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [16]:
combined_df= combined_data.copy()

In [17]:
df1 = combined_df.dropna(subset=['60_in_15mob'])

In [18]:
df1.shape

(123367, 2831)

In [19]:
df1['user_segment'].value_counts()

user_segment
OFFUS    90704
ONUS     32663
Name: count, dtype: int64

In [20]:
df = df1.copy()

In [21]:
df1.shape

(123367, 2831)

In [22]:
df1['unique_reference_number'].nunique()

123367

In [23]:
# change the month year whatever vintage you want test
df = df[df['month_year']=='2023-06']

In [24]:
df.shape

(5243, 2831)

In [27]:
data = df.copy()
data.replace([np.inf, -np.inf], np.nan, inplace=True)


/tmp/ipykernel_3036582/3786438119.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace([np.inf, -np.inf], np.nan, inplace=True)


In [28]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [29]:
model_df1.shape, oot_df1.shape

((0, 2831), (5243, 2831))

In [43]:
features_list = [
'num_PERSONAL LOAN_enq_last_6M',
'num_CONSUMER LOAN_enq_last_6M_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'num_CREDIT CARD_enq_last_12M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'Utilization_CREDIT CARD_opened_last_36M',
'total_emi',
'total_credit_all_open',
    
# banking
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',

#new banking features
'ratio_balance_after_2_days_max_credit_M3',
'ratio_5th_day_balance_to_max_credit_M3',
                 
#Loan application Features
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',
    
    
#ID Features
'aggs907',
'dm216s',
'dm206s',
'balmag01',
'g406s',

# New BUreau features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn',             
'month_year',
'60_in_15mob']

In [44]:
model_df = model_df1[features_list]

In [45]:
oot_df = oot_df1[features_list]

In [46]:
model_df.shape, oot_df.shape

((0, 40), (5243, 40))

In [47]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [48]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [49]:
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3036582/3109673050.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [50]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3036582/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [51]:
X_oot.shape,y_oot.shape

((5243, 38), (5243,))

In [52]:
# # Create DMatrix for OOT predictions
# if records available on OOT split use this cell else use below cell
data_x_pred = xgb.DMatrix(X_oot)
X_oot['60_in_15mob'] = y_oot
X_oot['y_pred'] = loaded_model.predict(data_x_pred)

In [ ]:
# #if records available on model split use this cell else use above cell
# data_x_pred = xgb.DMatrix(X_model)
# X_model['60_in_15mob'] = y_model
# X_model['month_year'] = model_df.loc[model_df.index,'month_year'].values

# X_model['y_pred'] = loaded_model.predict(data_x_pred)

In [53]:

# summary,auc, gini, ks = decile_analysis(X_model, 'y_pred', '60_in_15mob') 
summary,auc, gini, ks = decile_analysis(X_oot, 'y_pred', '60_in_15mob') 

In [54]:
print(f"AUC: {auc:.2f}, Gini: {gini:.2f}, KS: {ks:.2f}")

AUC: 0.72, Gini: 0.45, KS: 0.33


In [56]:
summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.291429         0.582646        0.810195        0.491353    525   
9          0.200382         0.446046        0.491094        0.407834    524   
8          0.141221         0.377919        0.407584        0.350276    524   
7          0.091603         0.328513        0.350241        0.309489    524   
6          0.106870         0.291607        0.309464        0.275155    524   
5          0.083810         0.260916        0.275016        0.246519    525   
4          0.066794         0.232746        0.246490        0.218640    524   
3          0.043893         0.202322        0.218629        0.188075    524   
2          0.024809         0.172206        0.188009        0.157483    524   
1          0.022857         0.134836        0.157341        0.079079    525   

        label_good  label_bad  
Decile                         
10             372        153  
9              419        105  
8              450         74  
7              476         48  
6              468         56  
5              481         44  
4              489         35  
3              501         23  
2              511         13  
1              513         12

In [57]:
feature_importance = loaded_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
35                                         days_on_pg       273.0
25                                              g406s       208.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       184.0
30                           No of enq in past 90days       150.0
19  deviation_from_median_income/standard_deviatio...       128.0
24                                           balmag01       116.0
26                         percentage_ever_delinquent       112.0
11                         bounced_transactions_count       108.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...       102.0
31                          Num_Enq_non_BANK_last_12M       101.0
6                highest_sanction_amount_credit_cards        97.0
12                           Min_Balance_Min_M1_M2_M3        93.0
8             Utilization_CREDIT CARD_opened_last_36M        90.0
1                   num_CONSUMER LOAN_enq_last_6M_36M        89.0
34  MEAN(r

In [58]:
feature_importance_df

Feature  Importance
35                                         days_on_pg       273.0
25                                              g406s       208.0
2              Num_of_PERSONAL LOAN_opened_last6M_12M       184.0
30                           No of enq in past 90days       150.0
19  deviation_from_median_income/standard_deviatio...       128.0
24                                           balmag01       116.0
26                         percentage_ever_delinquent       112.0
11                         bounced_transactions_count       108.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...       102.0
31                          Num_Enq_non_BANK_last_12M       101.0
6                highest_sanction_amount_credit_cards        97.0
12                           Min_Balance_Min_M1_M2_M3        93.0
8             Utilization_CREDIT CARD_opened_last_36M        90.0
1                   num_CONSUMER LOAN_enq_last_6M_36M        89.0
34  MEAN(raw.amount WHERE device_os_category = Win...        83.0
21                                            aggs907        82.0
4                                     Utilization_all        78.0
9                                           total_emi        75.0
7                          Age_of_oldest_open_account        73.0
3                        num_CREDIT CARD_enq_last_12M        69.0
33                                  distinct_merchant        65.0
37                              amt_first_android_txn        56.0
20                loan_amount_to_income#last_365_Days        54.0
23                                             dm206s        50.0
36  NUM_UNIQUE(pg_success_raw_data.merchant_id WHE...        47.0
13                               num_credit_gt_salary        46.0
28           months_since_most_recent_delinquency_all        45.0
10                              total_credit_all_open        44.0
32                     Num_of_installment_loan_opened        41.0
27   months_since_most_recent_delinquency_installment        39.0
17           ratio_balance_after_2_days_max_credit_M3        34.0
18             ratio_5th_day_balance_to_max_credit_M3        32.0
22                                             dm216s        31.0
29                       percentage_utilization_gt_75        29.0
16               ratio_total_debit_to_total_credit_M3        21.0
14                                    upi_trans_count        21.0
15                            Max_Credit_Avg_M1_M2_M3        20.0
0                       num_PERSONAL LOAN_enq_last_6M        14.0